In [1]:
#importing the libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
from selenium.webdriver.firefox.options import Options
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.action_chains import ActionChains

In [11]:
link = 'https://www.finn.com/de-DE/subscribe' #main link

In [4]:
response = requests.get(link)
soup = BeautifulSoup(response.text, 'html.parser')
response.status_code

200

In [4]:
print(soup)

<!DOCTYPE html>
<html lang="de-DE"><head><meta content="minimum-scale=1, initial-scale=1, width=device-width, shrink-to-fit=no" name="viewport"/><meta charset="utf-8"/><link href="/assets/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="/assets/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/assets/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/site.webmanifest" rel="manifest"/><link color="#5bbad5" href="/assets/favicon/safari-pinned-tab.svg" rel="mask-icon"/><meta content="#da532c" name="msapplication-TileColor"/><meta content="#ffffff" name="theme-color"/><link as="font" crossorigin="" href="/fonts/SuisseIntl/SuisseIntl-SemiBold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="" href="/fonts/Inter/Inter-Light.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="" href="/fonts/Inter/Inter-Medium.woff2" rel="preload" type="font/woff2"/><link as="f

In [5]:
soup.find('div', {'data-testid': 'product-card'})

<div data-testid="product-card"><a href="/de-DE/pdp/audi-rsetrongt-3131-tangorotmetallic" target="_blank"><div class="jss290"><div class="MuiBox-root jss296 jss293"><div class="MuiBox-root jss300 jss297"><div class="MuiBox-root jss301 jss299" style="background-color:#8F1E1E"></div><div class="MuiBox-root jss302 jss299" style="background-color:#000000"></div><div class="MuiBox-root jss303 jss299" style="background-color:#1D478A"></div></div></div><div class="jss294"><div class="jss295"><div class="jss304"><p class="MuiTypography-root jss305 MuiTypography-body2">Reduziert</p></div></div><div class="MuiBox-root jss309 jss306"><img alt="Audi RS e-tron GT" class="jss307" sizes="296px" src="https://res.cloudinary.com/finn-auto/image/fetch/q_auto,f_auto,c_fill,dpr_2,w_296,h_176,fl_progressive/https://dataops-nocodb-s3-bucket-production.s3.eu-central-1.amazonaws.com/nc/uploads/noco/fleet_blue_dragon/car_images/image/U1mT1o.png" srcset="https://res.cloudinary.com/finn-auto/image/fetch/q_auto,f_

In [6]:
soup.find_all('a' , href = True) 

[<a class="jss54" href="/de-DE"><svg aria-label="finn-logo" fill="none" height="26" viewbox="0 0 75 26" width="75" xmlns="http://www.w3.org/2000/svg"><path clip-rule="evenodd" d="M0 26V0H16.604V6.2452H6.31922V10.8616H14.2603V17.1068H6.31922V26H0ZM26.4537 0H20.1345V26H26.4537V0ZM44.4027 13.1899V0H50.7219V26H45.6487L36.2935 11.8309V26H29.9842V0H35.7694L44.4027 13.1899ZM68.6808 13.1899V0H75V26H69.9169L60.5716 11.8309V26H54.2524V0H60.0475L68.6808 13.1899Z" fill="#191919" fill-rule="evenodd"></path></svg></a>,
 <a class="jss61" href="/de-DE/subscribe"><p class="MuiTypography-root jss62 MuiTypography-body1">Auto finden</p></a>,
 <a href="/de-DE/subscribe" target="_self"><button class="MuiButtonBase-root MuiButton-root MuiButton-contained jss85 jss73 MuiButton-containedPrimary MuiButton-containedSizeLarge MuiButton-sizeLarge jss83 MuiButton-disableElevation" tabindex="0" type="button"><span class="MuiButton-label"><span class="">Alle Auto Abos entdecken</span></span></button></a>,
 <a class="

In [7]:
cars_hrefs = [a['href'] for a in soup.find_all('a', href=True) if '/pdp/' in a['href']] #extracting partially the url for each car
cars_hrefs

['/de-DE/pdp/audi-rsetrongt-3131-tangorotmetallic',
 '/de-DE/pdp/bmw-x4-3622-saphirschwarzmetallic',
 '/de-DE/pdp/jeep-avenger-10075-granitegreymetallic',
 '/de-DE/pdp/bmw-4ergrancoupe-3676-saphirschwarzmetallic',
 '/de-DE/pdp/mazda-6kombi-421-jetblackmetallic',
 '/de-DE/pdp/opel-corsaelectric-10511-arktisweissuni',
 '/de-DE/pdp/bmw-8ercoupem-3450-saphirschwarzmetallic',
 '/de-DE/pdp/audi-rsq3sportback-3114-mythosschwarzmetallic',
 '/de-DE/pdp/bmw-8ercoupe-3493-saphirschwarzmetallic',
 '/de-DE/pdp/fiat-500e-494-dreischicht-lackierunghimmelblau',
 '/de-DE/pdp/bmw-3ertouringm-3613-saphirschwarzmetallic',
 '/de-DE/pdp/nissan-townstar-505-urbangreyuni',
 '/de-DE/pdp/opel-grandland-3217-rubinrot',
 '/de-DE/pdp/opel-corsa-3195-dynamikorange',
 '/de-DE/pdp/opel-corsa-3209-dynamikorange',
 '/de-DE/pdp/opel-corsa-3212-kardiorot',
 '/de-DE/pdp/jeep-compass-2904-graphitegrey',
 '/de-DE/pdp/opel-grandland-3218-jadeweiss',
 '/de-DE/pdp/opel-corsa-3213-kardiorot',
 '/de-DE/pdp/alfaromeo-giulia-10181

### we have to use selenium to get all the car hrefs

In [12]:
geckodriver_path = r"C:\Users\Giannis Anastaso\Desktop\Useful tools\geckodriver.exe"
service = Service(geckodriver_path, port=0) 

In [17]:
time.sleep(2)
try:
    #driver.quit()  # Close the existing WebDriver instance
    driver = webdriver.Firefox(service=service)
    driver.get(link)

    # Scroll down to load more content
    for _ in range(10):  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  

    # Explicitly wait for the page to load after scrolling
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Get the updated HTML after scrolling
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # Extract hrefs
    cars_hrefs = [a['href'] for a in soup.find_all('a', href=True) if '/pdp/' in a['href']]
    print(cars_hrefs)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

['/de-DE/pdp/audi-rsetrongt-3131-tangorotmetallic', '/de-DE/pdp/bmw-x4-3622-saphirschwarzmetallic', '/de-DE/pdp/jeep-avenger-10075-granitegreymetallic', '/de-DE/pdp/bmw-4ergrancoupe-3676-saphirschwarzmetallic', '/de-DE/pdp/mazda-6kombi-421-jetblackmetallic', '/de-DE/pdp/opel-corsaelectric-10511-arktisweissuni', '/de-DE/pdp/bmw-8ercoupem-3450-saphirschwarzmetallic', '/de-DE/pdp/audi-rsq3sportback-3114-mythosschwarzmetallic', '/de-DE/pdp/bmw-8ercoupe-3493-saphirschwarzmetallic', '/de-DE/pdp/fiat-500e-494-dreischicht-lackierunghimmelblau', '/de-DE/pdp/bmw-3ertouringm-3613-saphirschwarzmetallic', '/de-DE/pdp/nissan-townstar-505-urbangreyuni', '/de-DE/pdp/opel-grandland-3217-rubinrot', '/de-DE/pdp/opel-corsa-3195-dynamikorange', '/de-DE/pdp/opel-corsa-3209-dynamikorange', '/de-DE/pdp/opel-corsa-3212-kardiorot', '/de-DE/pdp/opel-grandland-3218-jadeweiss', '/de-DE/pdp/opel-corsa-3213-kardiorot', '/de-DE/pdp/alfaromeo-giulia-10181-biancobanchiseuni', '/de-DE/pdp/opel-corsa-3211-kristallsilber'

In [18]:
len(cars_hrefs)

142

In [19]:
cars_hrefs[0]

'/de-DE/pdp/audi-rsetrongt-3131-tangorotmetallic'

In [24]:
car_links = []
for car in cars_hrefs:
    car_link = link[:20] + car
    car_links.append(car_link) #creating the final car links

In [25]:
car_links

['https://www.finn.com/de-DE/pdp/audi-rsetrongt-3131-tangorotmetallic',
 'https://www.finn.com/de-DE/pdp/bmw-x4-3622-saphirschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/jeep-avenger-10075-granitegreymetallic',
 'https://www.finn.com/de-DE/pdp/bmw-4ergrancoupe-3676-saphirschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/mazda-6kombi-421-jetblackmetallic',
 'https://www.finn.com/de-DE/pdp/opel-corsaelectric-10511-arktisweissuni',
 'https://www.finn.com/de-DE/pdp/bmw-8ercoupem-3450-saphirschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/audi-rsq3sportback-3114-mythosschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/bmw-8ercoupe-3493-saphirschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/fiat-500e-494-dreischicht-lackierunghimmelblau',
 'https://www.finn.com/de-DE/pdp/bmw-3ertouringm-3613-saphirschwarzmetallic',
 'https://www.finn.com/de-DE/pdp/nissan-townstar-505-urbangreyuni',
 'https://www.finn.com/de-DE/pdp/opel-grandland-3217-rubinrot',
 'https://www.finn.com/de-DE/pdp/opel-

In [27]:
# testing to get the fuel and transmission of one car

try:
    driver.quit()  # Close the existing WebDriver instance
    driver = webdriver.Firefox(service=service) 
    # Open the URL with Selenium
    driver.get(car_links[0])

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[contains(@class, "MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3")]'))
    )

    # Extract page source
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Initialize variables
    fuel_value = None
    transmission_value = None

    # Find the elements with the common class
    common_elements = soup.find_all(class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3')

    # Iterate through the common elements and extract information
    for element in common_elements:
        # Check if 'Kraftstoff' is present in the element
        if 'Kraftstoff' in element.get_text():
            fuel_index = element.get_text().find('Kraftstoff')
            fuel_value = element.get_text()[fuel_index + len('Kraftstoff'):].strip()

        # Check if 'Schaltung' is present in the element
        if 'Schaltung' in element.get_text():
            transmission_index = element.get_text().find('Schaltung')
            transmission_value = element.get_text()[transmission_index + len('Schaltung'):].strip()

    # Print or use the extracted information
    print("Fuel:", fuel_value)
    print("Transmission:", transmission_value)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Fuel: Elektro
Transmission: Automatik


In [31]:
car_links[100]
MuiTypography-root jss280 MuiTypography-h4

'https://www.finn.com/de-DE/pdp/bmw-x4-3704-sophistograubrillanteffektmetallic'

In [68]:
try:
    driver.quit()  # Close the existing WebDriver instance
    driver = webdriver.Firefox(service=service)    
    driver.get(car_links[75])

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[contains(@class, "MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3")]'))
    )

    # Extract page source
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the first h1 element on the page
    h1_element = soup.find('h1')

    # If h1_element is found, extract brand and model
    if h1_element:
        match = re.match(r'(Alfa Romeo|Audi|BMW|Fiat|Jeep|Land Rover|MG|MINI|Mazda|Nissan|Opel|VW)([\d\D]+)', h1_element.get_text(strip=True))
        brand = match.group(1).strip() if match else None
        model = match.group(2).strip() if match else None
    else:
        brand, model = None, None

    
    fuel_value = None
    transmission_value = None
    # Find the elements with the common class
    common_elements = soup.find_all(class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3')

    # Iterate through the common elements and extract information
    for element in common_elements:
        
        if 'Kraftstoff' in element.get_text():
            fuel_index = element.get_text().find('Kraftstoff')
            fuel_value = element.get_text()[fuel_index + len('Kraftstoff'):].strip()

        
        if 'Schaltung' in element.get_text():
            transmission_index = element.get_text().find('Schaltung')
            transmission_value = element.get_text()[transmission_index + len('Schaltung'):].strip()

    
    print("Brand:", brand)
    print("Model:", model)
    print("Fuel:", fuel_value)
    print("Transmission:", transmission_value)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    
    driver.quit()


Brand: Audi
Model: A4 Avant
Fuel: Benzin
Transmission: Automatik


### Create a df out of a list of dictionaries with data for each car

In [72]:
car_data = []   
try:
    for car_link in car_links:
        driver.quit()  
        driver = webdriver.Firefox(service=service) 
        driver.get(car_link)

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[contains(@class, "MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3")]'))
        )

        # Extract page source
        page_source = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        
        h1_element = soup.find('h1')

        # If h1_element is found, extract brand and model 
        if h1_element:
            match = re.match(r'(Alfa Romeo|Audi|BMW|Fiat|Jeep|Land Rover|MG|MINI|Mazda|Nissan|Opel|VW)([\d\D]+)', h1_element.get_text(strip=True))
            brand = match.group(1).strip() if match else None
            model = match.group(2).strip() if match else None
        else:
            brand, model = None, None

        
        fuel_value = None
        transmission_value = None
        # Find the elements with the common class
        common_elements = soup.find_all(class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 MuiGrid-grid-md-4 MuiGrid-grid-lg-3')

        # Iterate through the common elements and extract information
        for element in common_elements:
            
            if 'Kraftstoff' in element.get_text():
                fuel_index = element.get_text().find('Kraftstoff')
                fuel_value = element.get_text()[fuel_index + len('Kraftstoff'):].strip()

            
            if 'Schaltung' in element.get_text():
                transmission_index = element.get_text().find('Schaltung')
                transmission_value = element.get_text()[transmission_index + len('Schaltung'):].strip()

        # Append data for the current car to the list
        car_data.append({
            'Car link': car_link,
            'Brand': brand,
            'Model': model,
            'Fuel': fuel_value,
            'Transmission': transmission_value
        })

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    
    driver.quit()

# Create a DataFrame from the list of car data
car_df = pd.DataFrame(car_data)
car_df.head()

,Car link,Brand,Model,Fuel,Transmission
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik


In [76]:
car_df.head(20)

,Car link,Brand,Model,Fuel,Transmission
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik
5,https://www.finn.com/de-DE/pdp/opel-corsaelect...,Opel,Corsa Electric,Elektro,Automatik
6,https://www.finn.com/de-DE/pdp/bmw-8ercoupem-3...,BMW,8er Coupé M,Benzin,Automatik
7,https://www.finn.com/de-DE/pdp/audi-rsq3sportb...,Audi,RS Q3 Sportback,Benzin,Automatik
8,https://www.finn.com/de-DE/pdp/bmw-8ercoupe-34...,BMW,8er Coupé,Benzin,Automatik
9,https://www.finn.com/de-DE/pdp/fiat-500e-494-d...,Fiat,500e,Elektro,Automatik


In [77]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Car link      142 non-null    object
 1   Brand         142 non-null    object
 2   Model         142 non-null    object
 3   Fuel          142 non-null    object
 4   Transmission  142 non-null    object
dtypes: object(5)
memory usage: 5.7+ KB


In [78]:
car_df['Fuel'].value_counts()

Benzin            109
Diesel             23
Elektro             7
Plug-In-Hybrid      3
Name: Fuel, dtype: int64

In [80]:
car_df

,Car link,Brand,Model,Fuel,Transmission
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik
...,...,...,...,...,...
137,https://www.finn.com/de-DE/pdp/audi-q8-2274-my...,Audi,Q8,Plug-In-Hybrid,Automatik
138,https://www.finn.com/de-DE/pdp/audi-rs6avant-3...,Audi,RS 6 Avant,Benzin,Automatik
139,https://www.finn.com/de-DE/pdp/audi-q3sportbac...,Audi,Q3 Sportback,Diesel,Automatik
140,https://www.finn.com/de-DE/pdp/audi-q8-2266-my...,Audi,Q8,Plug-In-Hybrid,Automatik


In [79]:
#import the necessary library and export the df in an excel file

import openpyxl

car_df.to_excel('Finn_scraped_data.xlsx', index=False, sheet_name='Finn_Car_info')

### Create a copy of the df in order to transform it in the same form of the other dfs and integrate it correctly

In [96]:
working_df = car_df.copy(deep=False)
working_df

,Car link,Brand,Model,Fuel,Transmission
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik
...,...,...,...,...,...
137,https://www.finn.com/de-DE/pdp/audi-q8-2274-my...,Audi,Q8,Plug-In-Hybrid,Automatik
138,https://www.finn.com/de-DE/pdp/audi-rs6avant-3...,Audi,RS 6 Avant,Benzin,Automatik
139,https://www.finn.com/de-DE/pdp/audi-q3sportbac...,Audi,Q3 Sportback,Diesel,Automatik
140,https://www.finn.com/de-DE/pdp/audi-q8-2266-my...,Audi,Q8,Plug-In-Hybrid,Automatik


In [97]:
working_df.rename(columns={'Brand': 'brand', 'Model': 'model', 'Fuel': 'fuel', 'Transmission': 'gearbox'}, inplace=True)

In [98]:
working_df.rename(columns={'Car link': 'link'}, inplace=True)
working_df

,link,brand,model,fuel,gearbox
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik
...,...,...,...,...,...
137,https://www.finn.com/de-DE/pdp/audi-q8-2274-my...,Audi,Q8,Plug-In-Hybrid,Automatik
138,https://www.finn.com/de-DE/pdp/audi-rs6avant-3...,Audi,RS 6 Avant,Benzin,Automatik
139,https://www.finn.com/de-DE/pdp/audi-q3sportbac...,Audi,Q3 Sportback,Diesel,Automatik
140,https://www.finn.com/de-DE/pdp/audi-q8-2266-my...,Audi,Q8,Plug-In-Hybrid,Automatik


In [99]:
working_df['leasing_company'] = 'Finn'
working_df['price'] = np.nan
working_df['cc'] = np.nan
working_df['horsepower'] = np.nan
working_df['consumption'] = np.nan
working_df.head()

,link,brand,model,fuel,gearbox,leasing_company,price,cc,horsepower,consumption
0,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...,Audi,RS e-tron GT,Elektro,Automatik,Finn,NaN,NaN,NaN,NaN
1,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...,BMW,X4,Benzin,Automatik,Finn,NaN,NaN,NaN,NaN
2,https://www.finn.com/de-DE/pdp/jeep-avenger-10...,Jeep,Avenger,Benzin,Manuell,Finn,NaN,NaN,NaN,NaN
3,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...,BMW,4er Gran Coupé,Benzin,Automatik,Finn,NaN,NaN,NaN,NaN
4,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...,Mazda,6 Kombi,Benzin,Automatik,Finn,NaN,NaN,NaN,NaN


In [100]:
working_df = working_df[['leasing_company','brand','model','price','gearbox','fuel','cc','horsepower','consumption','link']]
working_df.head()

,leasing_company,brand,model,price,gearbox,fuel,cc,horsepower,consumption,link
0,Finn,Audi,RS e-tron GT,NaN,Automatik,Elektro,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...
1,Finn,BMW,X4,NaN,Automatik,Benzin,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...
2,Finn,Jeep,Avenger,NaN,Manuell,Benzin,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/jeep-avenger-10...
3,Finn,BMW,4er Gran Coupé,NaN,Automatik,Benzin,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...
4,Finn,Mazda,6 Kombi,NaN,Automatik,Benzin,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...


In [101]:
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   leasing_company  142 non-null    object 
 1   brand            142 non-null    object 
 2   model            142 non-null    object 
 3   price            0 non-null      float64
 4   gearbox          142 non-null    object 
 5   fuel             142 non-null    object 
 6   cc               0 non-null      float64
 7   horsepower       0 non-null      float64
 8   consumption      0 non-null      float64
 9   link             142 non-null    object 
dtypes: float64(4), object(6)
memory usage: 11.2+ KB


In [102]:
working_df['brand'] = working_df['brand'].astype(str).str.upper()
working_df['model'] = working_df['model'].astype(str).str.upper()
working_df['gearbox'] = working_df['gearbox'].astype(str)
working_df['fuel'] = working_df['fuel'].astype(str)

In [103]:
working_df['gearbox'].value_counts()

Automatik    130
Manuell       12
Name: gearbox, dtype: int64

In [104]:
working_df['gearbox'] = working_df['gearbox'].replace('Automatik', 'Automatic')
working_df['gearbox'] = working_df['gearbox'].replace('Manuell', 'Manual')

In [105]:
working_df['fuel'].value_counts()

Benzin            109
Diesel             23
Elektro             7
Plug-In-Hybrid      3
Name: fuel, dtype: int64

In [106]:
working_df['fuel'] = working_df['fuel'].replace('Benzin', 'Petrol')
working_df['fuel'] = working_df['fuel'].replace('Elektro', 'Electric')
working_df.head()

,leasing_company,brand,model,price,gearbox,fuel,cc,horsepower,consumption,link
0,Finn,AUDI,RS E-TRON GT,NaN,Automatic,Electric,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...
1,Finn,BMW,X4,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...
2,Finn,JEEP,AVENGER,NaN,Manual,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/jeep-avenger-10...
3,Finn,BMW,4ER GRAN COUPÉ,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...
4,Finn,MAZDA,6 KOMBI,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...


In [107]:
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   leasing_company  142 non-null    object 
 1   brand            142 non-null    object 
 2   model            142 non-null    object 
 3   price            0 non-null      float64
 4   gearbox          142 non-null    object 
 5   fuel             142 non-null    object 
 6   cc               0 non-null      float64
 7   horsepower       0 non-null      float64
 8   consumption      0 non-null      float64
 9   link             142 non-null    object 
dtypes: float64(4), object(6)
memory usage: 11.2+ KB


In [108]:
working_df['brand'].unique()

array(['AUDI', 'BMW', 'JEEP', 'MAZDA', 'OPEL', 'FIAT', 'NISSAN',
       'ALFA ROMEO', 'MG', 'VW', 'MINI'], dtype=object)

In [109]:
working_df['brand'] = working_df['brand'].replace('VW', 'VOLKSWAGEN')

In [110]:
working_df.head()

,leasing_company,brand,model,price,gearbox,fuel,cc,horsepower,consumption,link
0,Finn,AUDI,RS E-TRON GT,NaN,Automatic,Electric,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/audi-rsetrongt-...
1,Finn,BMW,X4,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-x4-3622-sap...
2,Finn,JEEP,AVENGER,NaN,Manual,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/jeep-avenger-10...
3,Finn,BMW,4ER GRAN COUPÉ,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/bmw-4ergrancoup...
4,Finn,MAZDA,6 KOMBI,NaN,Automatic,Petrol,NaN,NaN,NaN,https://www.finn.com/de-DE/pdp/mazda-6kombi-42...


In [111]:
working_df.to_excel('Finn_scraped_data_v2.xlsx', index=False, sheet_name='Finn_Car_info')